In [3]:
#Import the necessary libraries

import pandas as pd
import numpy as np
import datetime
import warnings

warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

%matplotlib inline

pd.set_option('display.max_columns', None)

In [4]:
#import the csv file

df = pd.read_csv('regression_data_clean.csv')   

### First examination of the data

In [5]:
df.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price
0,7129300520,10/13/14,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,221900
1,6414100192,12/9/14,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639,538000
2,5631500400,2/25/15,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062,180000
3,2487200875,12/9/14,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000,604000
4,1954400510,2/18/15,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503,510000


In [6]:
df.dtypes

id                 int64
date              object
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
price              int64
dtype: object

In [5]:
#sqt_living = sqft_above + sqft_basement  |  Maybe later consider dropping the last two

In [7]:
#sqft_living15 - sqft_living'= change in house size
#Whenever there is a change in the hosue or lot size one would assume that renovations took place
#however as can be seen in row 2 there is no data entry for the yr_renovated even though the living sze changed
#one explanation could be that the data was intepretated wrongly. 
#On Kaeggle some have taken sqft_living as the living spce of the nearest 15 neighbors.
#It could also be that the remodeling in those cases is not understood as renovated.


df[['sqft_living','sqft_lot','sqft_living15','sqft_lot15','yr_renovated']]

,sqft_living,sqft_lot,sqft_living15,sqft_lot15,yr_renovated
0,1180,5650,1340,5650,0
1,2570,7242,1690,7639,1991
2,770,10000,2720,8062,0
3,1960,5000,1360,5000,0
4,1680,8080,1800,7503,0
...,...,...,...,...,...
21592,1530,1131,1530,1509,0
21593,2310,5813,1830,7200,0
21594,1020,1350,1020,2007,0
21595,1600,2388,1410,1287,0


In [25]:
df[['date','sqft_living','sqft_living15']]

#the selling price dependent on tha actual size during the date of the sale, need to keep that in mind
#create a new column with sqft_living_sale_date

,date,sqft_living,sqft_living15
0,10/13/14,1180,1340
1,12/9/14,2570,1690
2,2/25/15,770,2720
3,12/9/14,1960,1360
4,2/18/15,1680,1800
...,...,...,...
21592,5/21/14,1530,1530
21593,2/23/15,2310,1830
21594,6/23/14,1020,1020
21595,1/16/15,1600,1410


In [7]:
df.columns

Index(['id', 'date', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15', 'price'],
      dtype='object')

In [8]:
#weird bathroom sizes... would bin them

df['bathrooms'].value_counts()

2.50    5377
1.00    3851
1.75    3048
2.25    2047
2.00    1930
1.50    1445
2.75    1185
3.00     753
3.50     731
3.25     589
3.75     155
4.00     136
4.50     100
4.25      79
0.75      71
4.75      23
5.00      21
5.25      13
5.50      10
1.25       9
6.00       6
5.75       4
0.50       4
8.00       2
6.25       2
6.75       2
6.50       2
7.50       1
7.75       1
Name: bathrooms, dtype: int64

In [9]:
# d numbers are not unique.... 
#check if the entire row is a dublicate, and drop it

df['id'].value_counts()

795000620     3
1825069031    2
2019200220    2
7129304540    2
1781500435    2
             ..
7812801125    1
4364700875    1
3021059276    1
880000205     1
1777500160    1
Name: id, Length: 21420, dtype: int64

In [22]:
duplicatedrows = df[df.duplicated()]
duplicatedrows

#no duplicated rows. I assume the houses, whose id appear various times in the df, 
#were sold and then resold in the given time period

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price


In [11]:
df['zipcode'].value_counts()

98103    602
98038    589
98115    583
98052    574
98117    553
        ... 
98102    104
98010    100
98024     80
98148     57
98039     50
Name: zipcode, Length: 70, dtype: int64

In [12]:
df['lat'].value_counts()

47.6624    17
47.5322    17
47.6846    17
47.5491    17
47.6886    16
           ..
47.2495     1
47.3847     1
47.4142     1
47.4237     1
47.2715     1
Name: lat, Length: 5033, dtype: int64

In [13]:
df['long'].value_counts()

-122.290    115
-122.300    111
-122.362    104
-122.291    100
-122.372     99
           ... 
-121.403      1
-121.804      1
-121.726      1
-121.895      1
-121.893      1
Name: long, Length: 751, dtype: int64

In [14]:
#waterfront could be a boolean (would it make sense to change it?)
#could correlate with view == 4

In [20]:
df['view'].value_counts()
#why view is counted with 0-4? 


0    19475
2      961
3      510
1      332
4      319
Name: view, dtype: int64

In [ ]:
#getting an estimation of what the view grading is about
#checking for houses in google maps using the latitude and longitude

In [17]:
df.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price
0,7129300520,10/13/14,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,221900
1,6414100192,12/9/14,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639,538000
2,5631500400,2/25/15,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062,180000
3,2487200875,12/9/14,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000,604000
4,1954400510,2/18/15,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503,510000


In [18]:
df.dtypes.to_frame()

,0
id,int64
date,object
bedrooms,int64
bathrooms,float64
sqft_living,int64
sqft_lot,int64
floors,float64
waterfront,int64
view,int64
condition,int64


In [19]:
#family houses:
    
#https://www.google.de/maps/place/47%C2%B030'40.3%22N+122%C2%B015'25.2%22W/@47.5111335,-122.2570803,3a,75y,128.38h,91.47t/data=!3m7!1e1!3m5!1sjXmVpKn4tBh5e3xT9rMKkw!2e0!6s%2F%2Fgeo2.ggpht.com%2Fmaps%2Fphotothumb%2Ffd%2Fv1%3Fbpb%3DChEKD3NlYXJjaC5nd3MtcHJvZBIgChIJJQJWrnxCkFQRfzQIrKDExkYqCg0AAAAAFQAAAAAaBAhWEFY%26gl%3DDE!7i16384!8i8192!4m5!3m4!1s0x0:0x0!8m2!3d47.5112!4d-122.257

In [21]:
# check df['view'], where view is 4

df[(df['view'] == 4)]

https://www.google.de/maps/place/47%C2%B026'58.9%22N+122%C2%B022'30.0%22W/@47.4496473,-122.3747237,3a,75y,212.03h,72.4t/data=!3m7!1e1!3m5!1sUEAuuV_SdXjaUEGC1GJD2A!2e0!6s%2F%2Fgeo1.ggpht.com%2Fmaps%2Fphotothumb%2Ffd%2Fv1%3Fbpb%3DChEKD3NlYXJjaC5nd3MtcHJvZBIgChIJIUAlG-dEkFQRI97Z9C086KEqCg0AAAAAFQAAAAAaBQh4EOgC%26gl%3DDE!7i16384!8i8192!4m5!3m4!1s0x0:0x0!8m2!3d47.4497!4d-122.375
        
            
        

SyntaxError: invalid syntax (<ipython-input-21-8e030888028c>, line 5)

In [ ]:
df[(df['view'] == 3)]  
https://www.google.de/maps/place/47%C2%B033'10.4%22N+122%C2%B017'34.8%22W/@47.5529077,-122.2927216,3a,75y,35.03h,73.68t/data=!3m7!1e1!3m5!1sExzwWn9VgI0PuFYmgwTmJw!2e0!6s%2F%2Fgeo3.ggpht.com%2Fcbk%3Fpanoid%3DExzwWn9VgI0PuFYmgwTmJw%26output%3Dthumbnail%26cb_client%3Dsearch.gws-prod.gps%26thumb%3D2%26w%3D360%26h%3D120%26yaw%3D243.4101%26pitch%3D0%26thumbfov%3D100!7i16384!8i8192!4m5!3m4!1s0x0:0x0!8m2!3d47.5529!4d-122.293
        

In [ ]:
df[(df['view'] == 2)]  

https://www.google.de/maps/place/47%C2%B032'50.6%22N+122%C2%B017'42.0%22W/@47.5472891,-122.2947359,3a,75y,258.77h,80.08t/data=!3m7!1e1!3m5!1sfJTZIaPU6UdPSHGKq7iZWw!2e0!6s%2F%2Fgeo0.ggpht.com%2Fmaps%2Fphotothumb%2Ffd%2Fv1%3Fbpb%3DChEKD3NlYXJjaC5nd3MtcHJvZBIgChIJ5w8WZPtBkFQRKIRT7S4ugpEqCg0AAAAAFQAAAAAaBQh4EOgC%26gl%3DDE!7i13312!8i6656!4m5!3m4!1s0x0:0x0!8m2!3d47.5474!4d-122.295
        

In [ ]:
df['waterfront'].value_counts()

In [ ]:
df['floors'].value_counts()

In [ ]:
#Similiar approach with high houses to see if they are still individual houses or apartment houses

In [ ]:
#check out the houses that are very high

df[(df['floors'] == 3.5)]  

https://www.google.de/maps/place/47%C2%B037'34.7%22N+122%C2%B018'50.4%22W/@47.6263534,-122.3146284,3a,75y,96.57h,90t/data=!3m7!1e1!3m5!1sGy9sn-3xxINMGlkSUhIlyw!2e0!6s%2F%2Fgeo3.ggpht.com%2Fcbk%3Fpanoid%3DGy9sn-3xxINMGlkSUhIlyw%26output%3Dthumbnail%26cb_client%3Dsearch.gws-prod.gps%26thumb%3D2%26w%3D360%26h%3D120%26yaw%3D96.56864%26pitch%3D0%26thumbfov%3D100!7i16384!8i8192!4m5!3m4!1s0x0:0x0!8m2!3d47.6263!4d-122.314
        